In [1]:
from pathlib import Path
import csv
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# =========================
# User settings
# =========================
REF_PATH = Path(r"./reference/p1.png")       # reference 原图（固定）
REC_DIR  = Path(r"./img")                    # 恢复图像文件夹（批处理）
OUT_DIR  = Path(r"./eval_out")               # 输出目录（txt + csv）

# alignment search settings (translation only)
SEARCH_PX = 120   # 最大搜索范围（像素，full-res）。你目前dx/dy很小，建议80~150即可
DS = 4            # 下采样倍数（粗搜索用）。2或4都行

# which files to include
IMG_EXTS = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff"}

# =========================
# Metrics
# =========================
def invert_u8(g: np.ndarray) -> np.ndarray:
    return 255 - g

def psnr_u8(a: np.ndarray, b: np.ndarray) -> float:
    a = a.astype(np.float32)
    b = b.astype(np.float32)
    mse = np.mean((a - b) ** 2)
    if mse == 0:
        return float("inf")
    return 10.0 * np.log10((255.0 ** 2) / mse)

def snr_mse_style_u8(ref: np.ndarray, test: np.ndarray) -> float:
    # 图像域“基于MSE”的SNR（你之前用的口径）
    ref_f = ref.astype(np.float32)
    test_f = test.astype(np.float32)
    mse = np.mean((ref_f - test_f) ** 2)
    mu = float(np.mean(ref_f))
    if mse == 0:
        return float("inf")
    return 10.0 * np.log10((mu ** 2) / mse)

def ssim_u8(a: np.ndarray, b: np.ndarray) -> float:
    return float(ssim(a, b, data_range=255))

# =========================
# Alignment (sliding via NCC)
# =========================
def shift_image(img: np.ndarray, dx: int, dy: int) -> np.ndarray:
    M = np.float32([[1, 0, dx],
                    [0, 1, dy]])
    return cv2.warpAffine(
        img, M, (img.shape[1], img.shape[0]),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_REPLICATE
    )

def best_shift_by_ncc(ref: np.ndarray, rec: np.ndarray, search: int, ds: int):
    """
    用“滑动相关性(NCC)”找 rec 相对 ref 的最佳平移。
    返回: (dx, dy, ncc_score)
    """
    # downsample for coarse search
    ref_s = cv2.resize(ref, (ref.shape[1] // ds, ref.shape[0] // ds), interpolation=cv2.INTER_AREA)
    rec_s = cv2.resize(rec, (rec.shape[1] // ds, rec.shape[0] // ds), interpolation=cv2.INTER_AREA)

    # mild high-pass to reduce background bias / striping dominance
    ref_blur = cv2.GaussianBlur(ref_s, (5, 5), 0)
    rec_blur = cv2.GaussianBlur(rec_s, (5, 5), 0)
    ref_hp = cv2.subtract(ref_s, ref_blur)
    rec_hp = cv2.subtract(rec_s, rec_blur)

    h, w = ref_hp.shape

    # choose a large center patch as template (avoid borders)
    patch_h = int(h * 0.70)
    patch_w = int(w * 0.70)
    y0 = (h - patch_h) // 2
    x0 = (w - patch_w) // 2

    template = rec_hp[y0:y0 + patch_h, x0:x0 + patch_w]

    # search window in downsampled pixels
    s = max(1, search // ds)

    x_min = max(0, x0 - s)
    x_max = min(w - patch_w, x0 + s)
    y_min = max(0, y0 - s)
    y_max = min(h - patch_h, y0 + s)

    # crop stage from ref where template can slide
    stage = ref_hp[y_min:y_max + patch_h, x_min:x_max + patch_w]

    # NCC template matching
    res = cv2.matchTemplate(stage, template, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(res)

    best_x = x_min + max_loc[0]
    best_y = y_min + max_loc[1]

    dx_s = best_x - x0
    dy_s = best_y - y0

    dx = int(round(dx_s * ds))
    dy = int(round(dy_s * ds))

    return dx, dy, float(max_val)

def eval_one(ref: np.ndarray, rec: np.ndarray, search: int, ds: int):
    """
    对单张恢复图：自动比较 as_is vs inverted，做平移配准，输出 best 结果 + 两种模式的明细。
    """
    # ensure same size for full-frame metrics
    if rec.shape != ref.shape:
        rec_rs = cv2.resize(rec, (ref.shape[1], ref.shape[0]), interpolation=cv2.INTER_LINEAR)
    else:
        rec_rs = rec

    all_modes = []
    best = None

    for mode, rec_cand in [("as_is", rec_rs), ("inverted", invert_u8(rec_rs))]:
        dx, dy, ncc = best_shift_by_ncc(ref, rec_cand, search=search, ds=ds)
        rec_aligned = shift_image(rec_cand, dx, dy)

        metrics = {
            "mode": mode,
            "dx": dx,
            "dy": dy,
            "ncc": ncc,
            "PSNR": psnr_u8(ref, rec_aligned),
            "SNR":  snr_mse_style_u8(ref, rec_aligned),
            "SSIM": ssim_u8(ref, rec_aligned),
        }
        all_modes.append(metrics)

        if (best is None) or (metrics["SSIM"] > best["SSIM"]):
            best = metrics

    return best, all_modes

# =========================
# IO helpers
# =========================
def read_gray(path: Path) -> np.ndarray:
    img = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Failed to read image: {path}")
    return img.astype(np.uint8)

def save_txt(out_path: Path, ref_path: Path, rec_path: Path, best: dict, all_modes: list, search: int, ds: int):
    lines = []
    lines.append(f"Reference: {ref_path}")
    lines.append(f"Recovered:  {rec_path}")
    lines.append(f"Search(px): {search}   Downsample(ds): {ds}")
    lines.append("")
    lines.append("=== ALL MODES ===")
    for m in all_modes:
        lines.append(
            f"[{m['mode']}] dx={m['dx']}, dy={m['dy']}, ncc={m['ncc']:.6f}, "
            f"PSNR={m['PSNR']:.6f}, SNR={m['SNR']:.6f}, SSIM={m['SSIM']:.6f}"
        )
    lines.append("")
    lines.append("=== BEST (by SSIM) ===")
    lines.append(
        f"mode={best['mode']}, dx={best['dx']}, dy={best['dy']}, ncc={best['ncc']:.6f}, "
        f"PSNR={best['PSNR']:.6f}, SNR={best['SNR']:.6f}, SSIM={best['SSIM']:.6f}"
    )
    out_path.write_text("\n".join(lines), encoding="utf-8")

def main():
    OUT_DIR.mkdir(parents=True, exist_ok=True)

    ref = read_gray(REF_PATH)

    # gather images
    rec_paths = sorted([p for p in REC_DIR.rglob("*") if p.suffix.lower() in IMG_EXTS])
    if not rec_paths:
        raise ValueError(f"No images found in {REC_DIR} with extensions {sorted(IMG_EXTS)}")

    csv_path = OUT_DIR / "results.csv"
    rows = []

    for rec_path in rec_paths:
        try:
            rec = read_gray(rec_path)
            best, all_modes = eval_one(ref, rec, search=SEARCH_PX, ds=DS)

            # save per-image txt
            txt_name = rec_path.stem + ".txt"
            txt_path = OUT_DIR / txt_name
            save_txt(txt_path, REF_PATH, rec_path, best, all_modes, SEARCH_PX, DS)

            # record csv row (best only + also save both modes optionally)
            row = {
                "file": rec_path.name,
                "path": str(rec_path),
                "best_mode": best["mode"],
                "dx": best["dx"],
                "dy": best["dy"],
                "ncc": best["ncc"],
                "PSNR": best["PSNR"],
                "SNR": best["SNR"],
                "SSIM": best["SSIM"],
            }
            rows.append(row)

            print(f"[OK] {rec_path.name} -> BEST mode={best['mode']} SSIM={best['SSIM']:.4f} PSNR={best['PSNR']:.2f}")

        except Exception as e:
            # still record failure
            row = {
                "file": rec_path.name,
                "path": str(rec_path),
                "best_mode": "ERROR",
                "dx": "",
                "dy": "",
                "ncc": "",
                "PSNR": "",
                "SNR": "",
                "SSIM": "",
                "error": str(e),
            }
            rows.append(row)
            print(f"[ERR] {rec_path.name}: {e}")

    # write CSV
    fieldnames = sorted(set().union(*[r.keys() for r in rows]))
    with csv_path.open("w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        w.writerows(rows)

    print(f"\nSaved CSV: {csv_path}")
    print(f"Saved per-image TXT files to: {OUT_DIR}")

if __name__ == "__main__":
    main()


d:\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


[OK] 771_445.5.png -> BEST mode=inverted SSIM=0.1197 PSNR=8.18
[OK] 771_445.5_long.png -> BEST mode=inverted SSIM=0.0928 PSNR=7.86
[OK] 771_742.5.png -> BEST mode=inverted SSIM=0.0765 PSNR=9.33
[OK] 771_742.5_long.png -> BEST mode=inverted SSIM=0.0333 PSNR=7.45
[OK] ht8_445.5.png -> BEST mode=inverted SSIM=0.0345 PSNR=7.56
[OK] ht8_742.5.png -> BEST mode=inverted SSIM=0.2501 PSNR=10.41
[OK] lpda_445.5.png -> BEST mode=inverted SSIM=0.0571 PSNR=8.39
[OK] lpda_445.5_0db.png -> BEST mode=inverted SSIM=0.0930 PSNR=8.24
[OK] lpda_445.5_long.png -> BEST mode=inverted SSIM=0.0424 PSNR=6.13
[OK] lpda_445.5_long_0db.png -> BEST mode=inverted SSIM=0.0575 PSNR=7.11
[OK] lpda_742.5.png -> BEST mode=inverted SSIM=0.0430 PSNR=6.16
[OK] lpda_742.5_0db.png -> BEST mode=inverted SSIM=0.2480 PSNR=9.75
[OK] lpda_742.5_2.png -> BEST mode=as_is SSIM=0.0471 PSNR=4.66
[OK] lpda_742.5_long.png -> BEST mode=inverted SSIM=0.0423 PSNR=6.06
[OK] lpda_742.5_long_0db.png -> BEST mode=inverted SSIM=0.0564 PSNR=7.42
